Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [8]:
import os
import pickle
import tensorflow as tf

In [3]:
data_root = './dataset/'
pickle_file = os.path.join(data_root, 'noMNIST_sanit.pickle')

In [4]:
with open(pickle_file, 'rb') as f:
    data = pickle.load(f, encoding='bytes')
    print('Data dict\'s key:', data.keys())
    _train_dataset = data['train_dataset']
    _train_labels = data['train_labels']
    _valid_dataset = data['valid_dataset']
    _valid_labels = data['valid_labels']
    _test_dataset = data['test_dataset']
    _test_labels = data['test_labels']
    del data
    print('Train set', _train_dataset.shape, 'Labels:', _train_labels.shape)
    print('Valid set', _valid_dataset.shape, 'Labels:', _valid_labels.shape)
    print('Test set', _test_dataset.shape, 'Labels:', _test_labels.shape)

Data dict's key: dict_keys(['train_dataset', 'train_labels', 'valid_dataset', 'valid_labels', 'test_dataset', 'test_labels'])
Train set (200000, 28, 28) Labels: (200000,)
Valid set (8927, 28, 28) Labels: (8927,)
Test set (8736, 28, 28) Labels: (8736,)


- data as a flat matrix.
- labels as float one-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size*image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [6]:
train_dataset, train_labels = reformat(_train_dataset, _train_labels)
valid_dataset, valid_labels = reformat(_valid_dataset, _valid_labels)
test_dataset, test_labels = reformat(_test_dataset, _test_labels)

print('Train set', train_dataset.shape, 'Labels:',train_labels.shape)
print('Valid set', valid_dataset.shape, 'Labels:',valid_labels.shape)
print('Test set', test_dataset.shape, 'Labels:',test_labels.shape)

Train set (200000, 784) Labels: (200000, 10)
Valid set (8927, 784) Labels: (8927, 10)
Test set (8736, 784) Labels: (8736, 10)


In [47]:
train_labels[:2]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

Train a multinomial logistic regression using simple gradient descent.

In [27]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    # Input data
    # tf_train_dataset: (train_subset, 28*28)
    # tf_train_labels: (train_subset, num_labels)
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # weights: [28*28, num_labels]
    weights = tf.get_variable('weights', 
                              shape=[tf_train_dataset.get_shape()[-1],
                                     tf_train_labels.get_shape()[-1]],
                              initializer=tf.truncated_normal_initializer(0., 1.))
    # biases: [num_labels, ]
    biases = tf.get_variable('biases',
                             shape=[tf_train_labels.get_shape()[-1]],
                             initializer=tf.initializers.constant(0.0))
    
    # Training computation.
    # _logits: [train_subset, num_labels]
    _logits = tf.matmul(tf_train_dataset, weights) + biases
    _loss = tf.reduce_mean(
                         tf.nn.softmax_cross_entropy_with_logits_v2(
                            labels=tf_train_labels,
                            logits=_logits))
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(_loss)
    
    # Predictions for the training, validation, and test data.
    # Normalized exponential function: softmax
    # softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), axis)
    train_prediction = tf.nn.softmax(_logits)
    valid_logits = tf.matmul(tf_valid_dataset, weights) + biases
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = tf.matmul(tf_test_dataset, weights) + biases
    test_prediction = tf.nn.softmax(test_logits)
    

In [46]:
_logits.shape

TensorShape([Dimension(10000), Dimension(10)])

In [34]:
steps = 1000

def accuracy(predictions, labels):
    acc = np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0] * 100.
    return acc

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('[Tensorflow]: Initialized!')
    for step in range(steps):
        # Run the computations.
        _, loss, predictions = session.run([optimizer, _loss, train_prediction])
        if ((step+1) % 100 == 0):
            print('Loss at step %d: %f' % (step+1, loss))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))
    # tensor.eval() == session.run(tensor)
    print(test_prediction.eval())

[Tensorflow]: Initialized!
Loss at step 100: 2.266094
Training accuracy: 72.3%
Validation accuracy: 69.7%
Loss at step 200: 1.833823
Training accuracy: 75.0%
Validation accuracy: 72.1%
Loss at step 300: 1.596566
Training accuracy: 76.3%
Validation accuracy: 72.7%
Loss at step 400: 1.434920
Training accuracy: 77.2%
Validation accuracy: 73.2%
Loss at step 500: 1.314515
Training accuracy: 78.1%
Validation accuracy: 73.4%
Loss at step 600: 1.220287
Training accuracy: 78.5%
Validation accuracy: 73.8%
Loss at step 700: 1.143593
Training accuracy: 79.0%
Validation accuracy: 73.9%
Loss at step 800: 1.079304
Training accuracy: 79.5%
Validation accuracy: 74.3%
Loss at step 900: 1.024315
Training accuracy: 79.8%
Validation accuracy: 74.4%
Loss at step 1000: 0.976624
Training accuracy: 80.2%
Validation accuracy: 74.5%
Test accuracy: 81.8%
[[0.1        0.1        0.1        ... 0.1        0.1        0.1       ]
 [0.1        0.1        0.1        ... 0.1        0.1        0.97682446]
 [0.1        0.

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [48]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training.
    weights = tf.get_variable('weights',
                              shape= [tf_train_dataset.get_shape()[-1], 
                                      tf_train_labels.get_shape()[-1]],
                              initializer=tf.initializers.truncated_normal(0., 1.))
    biases = tf.get_variable('biases',
                             shape= [tf_train_labels.get_shape()[-1]],
                             initializer=tf.constant_initializer(0.0))
    
    # Training compution.
    _logits = tf.matmul(tf_train_dataset, weights) + biases
    _loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(
                        labels=tf_train_labels, logits=_logits))
    
    # Optimizer
    opt = tf.train.GradientDescentOptimizer(0.5).minimize(_loss)
    
    # Prediction for training, validation, and the test data.
    train_prediction = tf.nn.softmax(_logits)
    valid_logits = tf.matmul(tf_valid_dataset, weights) + biases
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = tf.matmul(tf_test_dataset, weights) + biases
    test_prediction = tf.nn.softmax(test_logits)

In [57]:
test_dataset.shape

(8736, 784)

In [55]:
steps = 10000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('[Tensorflow]: Initialized!')
    for step in range(steps):
        # For shuffle data.
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: batch_data, tf_train_labels: batch_labels
        }
        
        _, loss, predictions = session.run(
                                [opt, _loss, train_prediction],
                                feed_dict=feed_dict
                               )
        if ((step+1) % 500 == 0):
            # %4.5f : 输出结果占4个占位符，保留5位小数.
            print('Minibatch step: %d, loss: %4.5f, acc: %.1f%%.'
                  % (step+1, loss, accuracy(predictions, batch_labels)))
            print('Validation accuracy: %.1f%%.' % accuracy(valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%.' % accuracy(test_prediction.eval(), test_labels))

[Tensorflow]: Initialized!
Minibatch step: 500, loss: 1.15563, acc: 81.2%.
Validation accuracy: 75.3%.
Minibatch step: 1000, loss: 1.60892, acc: 75.8%.
Validation accuracy: 75.9%.
Minibatch step: 1500, loss: 1.45336, acc: 76.6%.
Validation accuracy: 76.3%.
Minibatch step: 2000, loss: 0.99298, acc: 77.3%.
Validation accuracy: 77.2%.
Minibatch step: 2500, loss: 1.30347, acc: 75.0%.
Validation accuracy: 77.3%.
Minibatch step: 3000, loss: 0.76087, acc: 82.0%.
Validation accuracy: 78.2%.
Minibatch step: 3500, loss: 1.01908, acc: 77.3%.
Validation accuracy: 77.9%.
Minibatch step: 4000, loss: 0.80510, acc: 80.5%.
Validation accuracy: 78.4%.
Minibatch step: 4500, loss: 0.59977, acc: 84.4%.
Validation accuracy: 78.9%.
Minibatch step: 5000, loss: 0.74360, acc: 81.2%.
Validation accuracy: 78.9%.
Minibatch step: 5500, loss: 1.02882, acc: 75.8%.
Validation accuracy: 79.0%.
Minibatch step: 6000, loss: 0.88711, acc: 75.8%.
Validation accuracy: 79.4%.
Minibatch step: 6500, loss: 0.90879, acc: 78.9%.
V

Minibatch step: 53000, loss: 0.48402, acc: 87.5%.
Validation accuracy: 81.7%.
Minibatch step: 53500, loss: 0.77000, acc: 80.5%.
Validation accuracy: 81.0%.
Minibatch step: 54000, loss: 0.55395, acc: 83.6%.
Validation accuracy: 81.7%.
Minibatch step: 54500, loss: 0.50613, acc: 86.7%.
Validation accuracy: 81.4%.
Minibatch step: 55000, loss: 0.66210, acc: 80.5%.
Validation accuracy: 81.1%.
Minibatch step: 55500, loss: 0.59907, acc: 84.4%.
Validation accuracy: 82.0%.
Minibatch step: 56000, loss: 0.65402, acc: 82.8%.
Validation accuracy: 80.8%.
Minibatch step: 56500, loss: 0.63148, acc: 78.1%.
Validation accuracy: 79.9%.
Minibatch step: 57000, loss: 0.70840, acc: 80.5%.
Validation accuracy: 82.0%.
Minibatch step: 57500, loss: 0.73944, acc: 82.0%.
Validation accuracy: 81.6%.
Minibatch step: 58000, loss: 0.72005, acc: 80.5%.
Validation accuracy: 81.6%.
Minibatch step: 58500, loss: 0.49274, acc: 83.6%.
Validation accuracy: 80.9%.
Minibatch step: 59000, loss: 0.42546, acc: 87.5%.
Validation acc

#### Hidden Nodes

In [59]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training.
    weights_1 = tf.get_variable('weights_1',
                              shape= [tf_train_dataset.get_shape()[-1], 
                                      num_hidden_nodes],
                              initializer=tf.initializers.truncated_normal(0., 1.))
    biases_1 = tf.get_variable('biases_1',
                             shape= [num_hidden_nodes],
                             initializer=tf.constant_initializer(0.0))

    weights_2 = tf.get_variable('weights_2',
                              shape= [num_hidden_nodes, tf_train_labels.shape[-1]],
                              initializer=tf.initializers.truncated_normal(0., 1.))
    biases_2 = tf.get_variable('biases_2',
                             shape= [tf_train_labels.shape[-1]],
                             initializer=tf.constant_initializer(0.0))
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
    # Training compution.
    _logits = tf.matmul(lay1_train, weights_2) + biases_2
    _loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(
                        labels=tf_train_labels, logits=_logits))
    
    # Optimizer
    opt = tf.train.GradientDescentOptimizer(0.5).minimize(_loss)
    
    # Prediction for training, validation, and the test data.
    train_prediction = tf.nn.softmax(_logits)
    valid_lay1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_logits = tf.matmul(valid_lay1, weights_2) + biases_2
    valid_prediction = tf.nn.softmax(valid_logits)
    test_lay1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_logits = tf.matmul(test_lay1, weights_2) + biases_2
    test_prediction = tf.nn.softmax(test_logits)

In [60]:
steps = 10000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('[Tensorflow]: Initialized!')
    for step in range(steps):
        # For shuffle data.
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: batch_data, tf_train_labels: batch_labels
        }
        
        _, loss, predictions = session.run(
                                [opt, _loss, train_prediction],
                                feed_dict=feed_dict
                               )
        if ((step+1) % 500 == 0):
            # %4.5f : 输出结果占4个占位符，保留5位小数.
            print('Minibatch step: %d, loss: %4.5f, acc: %.1f%%.'
                  % (step+1, loss, accuracy(predictions, batch_labels)))
            print('Validation accuracy: %.1f%%.' % accuracy(valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%.' % accuracy(test_prediction.eval(), test_labels))

[Tensorflow]: Initialized!
Minibatch step: 500, loss: 15.00341, acc: 82.8%.
Validation accuracy: 78.7%.
Minibatch step: 1000, loss: 11.16014, acc: 80.5%.
Validation accuracy: 76.5%.
Minibatch step: 1500, loss: 6.39131, acc: 78.9%.
Validation accuracy: 80.6%.
Minibatch step: 2000, loss: 4.29619, acc: 82.0%.
Validation accuracy: 81.2%.
Minibatch step: 2500, loss: 6.44977, acc: 80.5%.
Validation accuracy: 80.2%.
Minibatch step: 3000, loss: 4.09252, acc: 82.8%.
Validation accuracy: 81.7%.
Minibatch step: 3500, loss: 4.67767, acc: 82.0%.
Validation accuracy: 82.0%.
Minibatch step: 4000, loss: 1.70067, acc: 84.4%.
Validation accuracy: 82.2%.
Minibatch step: 4500, loss: 1.97848, acc: 87.5%.
Validation accuracy: 82.8%.
Minibatch step: 5000, loss: 2.08788, acc: 82.0%.
Validation accuracy: 82.2%.
Minibatch step: 5500, loss: 1.80888, acc: 85.2%.
Validation accuracy: 82.2%.
Minibatch step: 6000, loss: 2.78531, acc: 82.8%.
Validation accuracy: 82.1%.
Minibatch step: 6500, loss: 1.63075, acc: 84.4%.